In [3]:
!pip install selenium

     ---------------------------------------- 0.0/118.7 kB ? eta -:--:--
     -------------------------------------- 118.7/118.7 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
    --------------------------------------- 0.2/9.9 MB 7.3 MB/s eta 0:00:02
   - -------------------------------------- 0.5/9.9 MB 7.1 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/9.9 MB 6.9 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/9.9 MB 6.4 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/9.9 MB 6.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/9.9 MB 6.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.7/9.9 MB 7.2 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/9.9 MB 7.2 MB/s eta 0:00:02
   --------- -----

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
USERNAME = "USERNAME"
PASSWORD = "PASSWORD"
PAGE = 1
# Initialize Chrome WebDriver
driver = webdriver.Chrome()
 
# Log in to Glassdoor
def login_to_glassdoor():
    driver.get("https://www.glassdoor.com/profile/login_input.htm")
    # Wait for page to loa
    time.sleep(2)
    # Find the username and password fields, and input your credentials

    username_field = driver.find_element(By.ID,"inlineUserEmail")
    username_field.send_keys(USERNAME)
    time.sleep(1)
    username_field.submit()
    print("1")
    time.sleep(1)
    # Click the login button
    # login_button = driver.find_element(By.CSS_SELECTOR, "button.email-button")
    # login_button.click()
    print("2")
    time.sleep(1)
    password_field = driver.find_element(By.ID,"inlineUserPassword")
    password_field.send_keys(PASSWORD)
    print("3")
    time.sleep(1)
    login_button = driver.find_element(By.CSS_SELECTOR, "button[data-size-variant='default']")
    login_button.click()
    print("4")
    # Wait for login to complete
    time.sleep(2)

 

# Navigate to the reviews page
def navigate_to_reviews():
    driver.get("https://www.glassdoor.com/Reviews/Honeywell-Reviews-E28_P" + str(PAGE) + ".htm")
    # driver.get("https://www.glassdoor.com/Reviews/Honeywell-Reviews-E28.htm")
    
    # Wait for page to load
    time.sleep(2)
 

# Extract reviews and save to a txt file
def save_reviews_to_txt(a):
    reviews = []
    for _ in range(10):
        try:
            # Find all review elements
            review_elements = driver.find_elements(By.CLASS_NAME,"noBorder.empReview.cf.pb-0.mb-0")
            # pros = driver.find_elements(By.XPATH,'//span[@data-test="pros"]')
            # review_elements = pros
            print("TRY:",a)
            # print(pros)
            for review_element in review_elements:
                try:
                    div_element = driver.find_element(By.CSS_SELECTOR,"div.v2__EIReviewDetailsV2__continueReading.v2__EIReviewDetailsV2__clickable.v2__EIReviewDetailsV2__newUiCta.mb")
                    div_element.click()
                except:
                    pass
                time.sleep(0.1)
                # print("ADDING SHIT")
                # print(review_element)
                review_text = review_element.text
                reviews.append(review_text)
                time.sleep(0.05)
                # print(review_text)
                # break
            break
        except:
            continue

    # Save the reviews to a txt file

    with open(str(a) + "glassdoor_reviews.txt", "w", encoding="utf-8") as file:
        file.write("\n\n".join(reviews))
    print("Reviews saved to glassdoor_reviews.txt")

 

def next_page():
    max_retries = 20
    for a in range(max_retries):
        try:
            button = driver.find_element(By.CSS_SELECTOR,"button[aria-label='Next']")
            button.click()
            time.sleep(0.2)
            return
        except:
            try:
                button = driver.find_element(By.CLASS_NAME,"qual_x_close")
                button.click()
            except:
                pass
            continue
    raise ValueError('no next page code stupid')
# Execute the entire process
def main():
    login_to_glassdoor()
    navigate_to_reviews()
    for a in range(PAGE,1000000000):
        save_reviews_to_txt(a)
        next_page()
        time.sleep(1)
    driver.quit()
 
if __name__ == "__main__":
    main()

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
# Initialize the WebDriver
driver = webdriver.Chrome()  # or webdriver.Firefox(), or any other WebDriver you're using

# Open the webpage
driver.get("https://unabated.com/nba/props")
driver.execute_script("document.body.style.zoom='30%'")
time.sleep(2)
# Find the div element using XPath
parent_div = driver.find_element(By.CSS_SELECTOR, ".ag-center-cols-container")
# div_element = parent_div.find_elements(By.TAG_NAME, "div")
div_element = parent_div.find_elements(By.CSS_SELECTOR, '.ag-center-cols-container > div')
# child_divs = viewport_div.find_elements_by_css_selector('.ag-center-cols-container > div')
# print(div_element.text)
# print(div_element.get_attribute('row-id'))

for a in div_element:
    print(a.text)
    temp = a.get_attribute('row-id')
    if temp is not None:
        print(a.get_attribute('row-id'))

# Do something with the div_element, for example, get its text


# Close the WebDriver
driver.quit()

              o5.5 -128
u5.5 +100
    o5.5 -130
u5.5 +100
o5.5 -130
u5.5 +100
        o5.5 -130
u5.5 +100
    
4360-58269
o19.5 -102
u19.5 -130
o19.5 -102
u19.5 -130
  o19.5 -105
u19.5 -125
o19.5 -112
u19.5 -112
o18.5 -122
u18.5 +100
o19.5 -105
u19.5 -125
o19.5 -120
u19.5 -110
o19.5 +110
u19.5 -152
o19.5 -110
u19.5 -120
o18.5 -123
u18.5 -105
o18.5 -123
u18.5 -105
o19.5 -108
u19.5 -124
o19.5 -103
u19.5 -130
o19.5 -133
u19.5 -118
o19.5 -103
u19.5 -130
o18.5 -123
u18.5 -105
o19.5 -108
u19.5 -124
o21.5 +155
u21.5 -235
5424-58269
      o7.5 -115
u7.5 -115
o7.5 -112
u7.5 -112
o7.5 -105
u7.5 -115
o7.5 -110
u7.5 -120
o7.5 -111
u7.5 -118
  o7.5 -115
u7.5 -115
o7.5 -108
u7.5 -120
o7.5 -108
u7.5 -120
        o7.5 -108
u7.5 -120
    
5603-58269
                                      
42526-58269
o26.5 -115
u26.5 -115
o26.5 -115
u26.5 -116
  o26.5 -115
u26.5 -115
o27.5 -112
u27.5 -112
o26.5 -122
u26.5 +100
o26.5 -115
u26.5 -115
o26.5 -110
u26.5 -120
o26.5 -114
u26.5 -120
o26.5 -115
u26.5 -115
o27.5 

In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

def cleanpd(s):
    o = s.replace("  ", " ,").replace("\no", "\n,o").replace("\n ", "\n, ")
    return(o.split(","))

# Assuming you have set up your WebDriver, such as ChromeDriver
driver = webdriver.Chrome()

# Navigate to your webpage
driver.get("https://unabated.com/nba/props")
driver.execute_script("document.body.style.zoom='30%'")
time.sleep(0.3)

# Find the first div with class .ag-center-cols-viewport
viewport_div = driver.find_element(By.CSS_SELECTOR,'.ag-center-cols-viewport')
name_div = driver.find_element(By.CSS_SELECTOR,'.ag-pinned-left-cols-container')

# Keep track of the number of child divs found
last_child_count = 0
player_data = {}
id2name = {}

# Loop to keep scrolling down until no more new child divs are loaded
while True:
    # Find all child divs under the viewport div with the specified CSS selector
    child_divs = viewport_div.find_elements(By.CSS_SELECTOR, '.ag-center-cols-container > div')
    id_child = name_div.find_elements(By.CSS_SELECTOR, ".ag-pinned-left-cols-container > div")
    

    body = child_divs[-1]
    # Scroll down the page to load more content
    time.sleep(0.3)
    # Create an ActionChains object
    actions = ActionChains(driver)

    # Scroll the element into view
    actions.move_to_element(body)

    # Perform a scroll action (e.g., scroll down)
    try:
        actions.click_and_hold().move_by_offset(0, 100).release().perform()
    except:
        pass
    for index, div in enumerate(child_divs, start=1):
        # Get the text of the div
        try:
            temp = div.get_attribute('row-id')
            if temp is not None:
                # print(player_data)
                player_data[temp] = cleanpd(div.text)
        except:
            pass
    for index, div in enumerate(id_child, start=1):
        # Get the text of the div
        name_child = div.find_element(By.CSS_SELECTOR, 'div.d-flex.flex-column.align-items-start[style="font-size: 0.75rem; line-height: 24px;"]')
        try:
            id2name[div.get_attribute('row-id')] = name_child.text
            # print(name_child.text)
            # print(div.get_attribute('row-id'))
        except:
            pass
            
    if len(player_data.keys()) == last_child_count:
        break
    last_child_count = len(player_data.keys())
        # Print or process the text and attributes as needed


# Iterate through each child div to get its text and attributes


# Finally, remember to close the browser window when done
driver.quit()

NAMES
NAMES
{'42667-58278': [' ', 'o9.5 -126\nu9.5 -105\n', ' ', 'o9.5 -120\nu9.5 -110\n', ' ', 'o9.5 -125\nu9.5 -102\n', 'o9.5 -125\nu9.5 -105\n', 'o9.5 -125\nu9.5 -105\n', 'o9.5 -135\nu9.5 -101\n', 'o9.5 -120\nu9.5 -110\n', 'o9.5 -116\nu9.5 -111\n', 'o9.5 -116\nu9.5 -111\n', ' ', ' ', ' ', ' ', 'o9.5 -116\nu9.5 -111\n', ' ', ' ', ''], '42679-58278': [' ', 'o10.5 +100\nu10.5 -132\n', ' ', 'o10.5 -105\nu10.5 -125\n', ' ', 'o10.5 +100\nu10.5 -128\n', 'o10.5 +100\nu10.5 -130\n', 'o10.5 +100\nu10.5 -133\n', 'o10.5 -103\nu10.5 -133\n', ' ', 'o9.5 -130\nu9.5 +100\n', 'o9.5 -130\nu9.5 +100\n', ' ', ' ', 'o10.5 -105\nu10.5 -137\n', ' ', 'o9.5 -130\nu9.5 +100\n', ' ', ' ', ''], '42838-58278': [' ', 'o22.5 -112\nu22.5 -121\n', ' ', 'o22.5 -115\nu22.5 -115\n', ' ', 'o22.5 -108\nu22.5 -120\n', 'o22.5 -110\nu22.5 -120\n', 'o22.5 -111\nu22.5 -120\n', 'o22.5 -119\nu22.5 -115\n', 'o22.5 -110\nu22.5 -120\n', 'o22.5 -106\nu22.5 -122\n', 'o22.5 -106\nu22.5 -122\n', ' ', ' ', 'o22.5 -118\nu22.5 -125\n', 

In [69]:
q = 'Bookmaker, Sharp Book, Sporttrade, bet365, pick6, Fanduel, DraftKings, BetMGM, Caesars, POINTSBET, BETONLINE, BOVADA, UNIBET, Sugar House, 888sport, BETRivers, Sports Betting, FourWinds, parx'.split(",")
player_data["Source"] = q.split()
for a in player_data.keys():
    print(a, len(a))

42667-58278 11
42679-58278 11
42838-58278 11
43228-58278 11
43395-58278 11
43426-58278 11
43533-58278 11
238452-58278 12
241037-58278 12
242856-58278 12
42798-58279 11
42962-58279 11
43020-58279 11
43092-58279 11
43294-58279 11
43313-58279 11
43316-58279 11
43441-58279 11
242292-58279 12
243114-58279 12
1357-58280 10
42609-58280 11
42720-58280 11
42849-58280 11
43138-58280 11
43243-58280 11
43416-58280 11
43438-58280 11
43530-58280 11
417203-58280 12
42605-58282 11
42608-58282 11
42988-58282 11
43061-58282 11
43154-58282 11
43274-58282 11
43333-58282 11
43351-58282 11
43434-58282 11
43496-58282 11
240737-58282 12
42685-58281 11
42707-58281 11
43144-58281 11
43358-58281 11
43387-58281 11
43491-58281 11
43558-58281 11
42647-58283 11
42709-58283 11
43096-58283 11
43165-58283 11
43392-58283 11
43526-58283 11
242200-58283 12
417201-58283 12
Source 6


In [92]:
import pandas as pd

df = pd.DataFrame(player_data)

display(df.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
42667-58278,,o9.5 -126\nu9.5 -105\n,,o9.5 -120\nu9.5 -110\n,,o9.5 -125\nu9.5 -102\n,o9.5 -125\nu9.5 -105\n,o9.5 -125\nu9.5 -105\n,o9.5 -135\nu9.5 -101\n,o9.5 -120\nu9.5 -110\n,o9.5 -116\nu9.5 -111\n,o9.5 -116\nu9.5 -111\n,,,,,o9.5 -116\nu9.5 -111\n,,,
42679-58278,,o10.5 +100\nu10.5 -132\n,,o10.5 -105\nu10.5 -125\n,,o10.5 +100\nu10.5 -128\n,o10.5 +100\nu10.5 -130\n,o10.5 +100\nu10.5 -133\n,o10.5 -103\nu10.5 -133\n,,o9.5 -130\nu9.5 +100\n,o9.5 -130\nu9.5 +100\n,,,o10.5 -105\nu10.5 -137\n,,o9.5 -130\nu9.5 +100\n,,,
42838-58278,,o22.5 -112\nu22.5 -121\n,,o22.5 -115\nu22.5 -115\n,,o22.5 -108\nu22.5 -120\n,o22.5 -110\nu22.5 -120\n,o22.5 -111\nu22.5 -120\n,o22.5 -119\nu22.5 -115\n,o22.5 -110\nu22.5 -120\n,o22.5 -106\nu22.5 -122\n,o22.5 -106\nu22.5 -122\n,,,o22.5 -118\nu22.5 -125\n,,o22.5 -106\nu22.5 -122\n,,,
43228-58278,,o7.5 -146\nu7.5 +108\n,,o7.5 -140\nu7.5 +110\n,,o8.5 +104\nu8.5 -132\n,o7.5 -140\nu7.5 +110\n,o7.5 -143\nu7.5 +105\n,o7.5 -145\nu7.5 +106\n,o8.5 +100\nu8.5 -130\n,o8.5 +107\nu8.5 -139\n,o8.5 +107\nu8.5 -139\n,,,,,o8.5 +107\nu8.5 -139\n,,,
43395-58278,,o31.5 -110\nu31.5 -120\n,,o31.5 -125\nu31.5 -105\n,,o31.5 -113\nu31.5 -113\n,o31.5 -125\nu31.5 -105\n,o31.5 -111\nu31.5 -120\n,o31.5 -115\nu31.5 -119\n,o32.5 -110\nu32.5 -120\n,o31.5 -115\nu31.5 -112\n,o31.5 -115\nu31.5 -112\n,,,o31.5 -118\nu31.5 -125\n,,o31.5 -115\nu31.5 -112\n,,,
43426-58278,,o10.5 -127\nu10.5 -106\n,,o10.5 -120\nu10.5 -110\n,,o10.5 -113\nu10.5 -113\n,o10.5 -125\nu10.5 -105\n,o10.5 -125\nu10.5 -105\n,o10.5 -133\nu10.5 -103\n,o10.5 -125\nu10.5 -105\n,o10.5 -122\nu10.5 -106\n,o10.5 -122\nu10.5 -106\n,,,o10.5 -133\nu10.5 -110\n,,o10.5 -122\nu10.5 -106\n,,,
43533-58278,,o21.5 -127\nu21.5 -105\n,,o21.5 -120\nu21.5 -110\n,,o22.5 +100\nu22.5 -128\n,o21.5 -140\nu21.5 +110\n,o21.5 -125\nu21.5 -105\n,o21.5 -125\nu21.5 -109\n,o22.5 -115\nu22.5 -115\n,o21.5 -127\nu21.5 -102\n,o21.5 -127\nu21.5 -102\n,,,o21.5 -133\nu21.5 -110\n,,o21.5 -127\nu21.5 -102\n,,,
238452-58278,,o11.5 +100\nu11.5 -133\n,,,,o11.5 +104\nu11.5 -132\n,o11.5 +100\nu11.5 -130\n,,o11.5 -103\nu11.5 -133\n,o11.5 -110\nu11.5 -120\n,,,,,,,,,,
241037-58278,,o18.5 -124\nu18.5 -109\n,,o18.5 -120\nu18.5 -110\n,,o18.5 -108\nu18.5 -118\n,o18.5 -120\nu18.5 -110\n,o18.5 -120\nu18.5 -110\n,o18.5 -128\nu18.5 -106\n,o18.5 -115\nu18.5 -115\n,o18.5 -119\nu18.5 -109\n,o18.5 -119\nu18.5 -109\n,,,o18.5 -133\nu18.5 -118\n,,o18.5 -119\nu18.5 -109\n,,,
242856-58278,,o14.5 -164\nu14.5 +122\n,,o14.5 -145\nu14.5 +115\n,,o15.5 -108\nu15.5 -118\n,o14.5 -125\nu14.5 -105\n,o14.5 -125\nu14.5 -105\n,o14.5 -128\nu14.5 -106\n,o14.5 -125\nu14.5 -105\n,o14.5 -122\nu14.5 -106\n,o14.5 -122\nu14.5 -106\n,,,o14.5 -133\nu14.5 -111\n,,o14.5 -122\nu14.5 -106\n,,,


In [91]:
q = 'Bookmaker, Sharp Book, Sporttrade, bet365, pick6, Fanduel, DraftKings, BetMGM, Caesars, POINTSBET, BETONLINE, BOVADA, UNIBET, Sugar House, 888sport, BETRivers, Sports Betting, FourWinds, parx, '.split(",")

tempd = {}
for a in range(len(q)):
    tempd[a] = q[a]
tempd

{0: 'Bookmaker',
 1: ' Sharp Book',
 2: ' Sporttrade',
 3: ' bet365',
 4: ' pick6',
 5: ' Fanduel',
 6: ' DraftKings',
 7: ' BetMGM',
 8: ' Caesars',
 9: ' POINTSBET',
 10: ' BETONLINE',
 11: ' BOVADA',
 12: ' UNIBET',
 13: ' Sugar House',
 14: ' 888sport',
 15: ' BETRivers',
 16: ' Sports Betting',
 17: ' FourWinds',
 18: ' parx',
 19: ' '}

In [ ]:
combinedpd = {}
for a in player_data.keys():
    combinedpd[id2name[a]]= player_data[a]

df = pd.DataFrame(combinedpd)
df = df.rename(index=tempd, errors="raise")
# Splitting the rows


display(df.T)

In [157]:
import re
def extract_text(s):
    # Define a regular expression pattern to match the desired components
    pattern = r'o(\d+\.\d+)\s*(-?\d+)\nu(\d+\.\d+)\s*([-+]\d+)'

    # Use re.findall to find all occurrences of the pattern in the string
    matches = re.findall(pattern, s)

    # Extract the matched groups from each occurrence
    # results = [(float(match[0]), int(match[1]), match[2], int(match[3])) for match in matches]
    if len(matches) == 0:
        return []
    # print(matches[0])
    return [float(matches[0][0]),int(matches[0][1]),float(matches[0][2]),int(matches[0][3])]

def oavgc(row):
    o = {}
    o2 = {}
    for c in df.columns:
        if "o" in row[c]:
            results = extract_text(row[c])
            if len(results) != 4:
                continue
            if results[0] in o:
                o[results[0]].append(results[1])
            else:
                o[results[0]] = [results[1]]
    for p in o.keys():
        o2[p] = round(sum(o[p]) / len(o[p]),1)
    return o2

def uavgc(row):
    o = {}
    o2 = {}
    for c in df.columns:
        if "u" in row[c]:
            results = extract_text(row[c])
            if len(results) != 4:
                continue
            if results[0] in o:
                o[results[0]].append(results[3])
            else:
                o[results[0]] = [results[3]]
    for p in o.keys():
        o2[p] = round(sum(o[p]) / len(o[p]),1)
    return o2


In [159]:
df['Over'] = df.apply(oavgc, axis=1)
df['Under'] = df.apply(uavgc, axis=1)
df[["Over", "Under"]].to_csv('out.csv')


In [110]:
list(df.iloc[1])

['o9.5 -126\nu9.5 -105\n',
 'o10.5 +100\nu10.5 -132\n',
 'o22.5 -112\nu22.5 -121\n',
 'o7.5 -146\nu7.5 +108\n',
 'o31.5 -110\nu31.5 -120\n',
 'o10.5 -127\nu10.5 -106\n',
 'o21.5 -127\nu21.5 -105\n',
 'o11.5 +100\nu11.5 -133\n',
 'o18.5 -124\nu18.5 -109\n',
 'o14.5 -164\nu14.5 +122\n',
 'o13.5 -103\nu13.5 -129\n',
 'o14.5 -129\nu14.5 -103\n',
 ' ',
 'o22.5 -117\nu22.5 -113\n',
 'o29.5 +105\nu29.5 -141\n',
 ' ',
 'o22.5 -125\nu22.5 -106\n',
 'o19.5 -129\nu19.5 -104\n',
 'o20.5 -140\nu20.5 +106\n',
 'o12.5 +103\nu12.5 -137\n',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 'o12.5 -127\nu12.5 -106\n',
 'o23.5 -141\nu23.5 +107\n',
 'o26.5 -109\nu26.5 -122\n',
 'o13.5 -113\nu13.5 -119\n',
 'o11.5 -133\nu11.5 +100\n',
 'o19.5 -106\nu19.5 -125\n',
 ' ',
 'o29.5 +101\nu29.5 -137\n',
 'o10.5 -123\nu10.5 -108\n',
 'o10.5 -112\nu10.5 -121\n',
 'o19.5 -121\nu19.5 -111\n',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']